In [ ]:
from __future__ import unicode_literals, print_function, division
import string
import gc
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import nltk
nltk.download('punkt')
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import pickle
from scipy.stats import skew
import cv2
import math
from google.colab import files
from google.colab.patches import cv2_imshow
import numpy as np
from string import printable
from re import X
from IPython.core import formatters
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import xml.etree.ElementTree as ET
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from io import open
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?']+", r" ", s)
    return s

MAX_LENGTH =  300

In [ ]:
videos = ['/content/rachel1.mp4', '/content/1-Ben-Introduction-SD.mov']
xml = ['/content/xml_extract_Rachel_2012-01-27_sc51_9.xml', 'xml_extract_1-Ben-Introduction_17.xml']

sequences =  []
translatedSequences =  []
AllSigns = []
AllFrames = []
AllFramesUncompressed = []
Labels = []

for video, xml in zip(videos, xml):
  capture = cv2.VideoCapture(video)
  count = 0
  frames = []
  frameNr = 0
  
  while (True):
      success, frame = capture.read()
      if success:
          frame = cv2.resize(frame, (320,240), interpolation = cv2.INTER_AREA)
          frames.append(frame)
      else:
          break
      count = count+1
  capture.release()

  print(len(frames))

  tree = ET.parse(xml)
  
  # getting the parent tag of
  # the xml document
  root = tree.getroot()
  
  # printing the root (parent) tag
  # of the xml document, along with
  # its memory location
  print(root)
  
  # printing the attributes of the
  # first tag from the parent
  print(root[0].attrib)
  
  # printing the text contained within
  # first subtag of the 5th tag from
  # the parent
  print(root[0][0][3][0][0][0][2])

  utterances= tree.findall('.//UTTERANCE')
  signs = utterances[1][3].findall('.//SIGN')
  UtteranceSigns = utterances[1][3].findall('.//SIGN')

  for sign  in tree.findall('.//SIGN'):
    # check if exists in unique_list or not
    if sign[0].text not in AllSigns:
      AllSigns.append(sign[0].text)

  label_map1 = {num:label for num, label in enumerate(AllSigns)}
  label_map2 = {label:num for num, label in enumerate(AllSigns)}

  
  for utterance in utterances:
      UtteranceSigns = utterance[3].findall('.//SIGN')
      UtteranceSignsSorted = sorted(UtteranceSigns, key=lambda x: x[13].get('START_FRAME'))
      utteranceFrameSigns = []
      signIDs = []
      FrameSequence = []
      FrameSequenceUncompressed = []
      SignSequence = ""
      a=0
      for sign in UtteranceSignsSorted:
        if sign.get('ID') not in signIDs:
          signIDs.append(sign.get('ID'))
          for frame_num in range(int(int(sign[13].get('START_FRAME'))/1000),int(int(sign[13].get('END_FRAME'))/1000)):
            if (frame_num - (int(int(sign[13].get('START_FRAME'))/1000)) ) % 3 == 0:
              FrameSequence.append(frames[frame_num-1].flatten())
              FrameSequenceUncompressed.append(frames[frame_num-1])
              utteranceFrameSigns.append(label_map2[sign[0].text])
      Labels.append(utteranceFrameSigns)
      translatedSequences.append(normalizeString(utterance[0].text))
      sequences.append(normalizeString(SignSequence))
      AllFrames.append(FrameSequence)
      AllFramesUncompressed.append(FrameSequenceUncompressed)

6996
<Element 'SIGNSTREAM_DAI' at 0x7fba05110530>
{}
<Element 'UTTERANCES' at 0x7fba05110e90>
4409
<Element 'SIGNSTREAM_DAI' at 0x7fba05619a70>
{}
<Element 'UTTERANCES' at 0x7fba056197d0>


In [ ]:
!pip install mediapipe
import mediapipe as mp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=True,
                      max_num_hands=2,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)

In [ ]:
u = 0
x = 0
HandTrackingSequences = []
for utterance in AllFramesUncompressed:
  u = u +1
  print(u)
  HandTrackingUtterance = [] 

  for frame in utterance:
    landmarks = []

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame)

    count = 0
    if results.multi_hand_landmarks != None:
      for hand_landmarks in results.multi_hand_world_landmarks:
        count = count +1
        for landmark in hand_landmarks.landmark:
          landmarks.append(landmark.x)
          landmarks.append(landmark.y)
          landmarks.append(landmark.z)

      if count < 2:
        x= x+1
        print('yyyyyyyyyy')
        for i in range(21):
          landmarks.append(0)
          landmarks.append(0)
          landmarks.append(0)


    else:
      print('xxxxxxxxx')
      x= x+1
      for i in range(2):
        for i in range(21):
          landmarks.append(0)
          landmarks.append(0)
          landmarks.append(0)
        
    HandTrackingUtterance.append(landmarks)
  HandTrackingSequences.append(HandTrackingUtterance)

1
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
2
yyyyyyyyyy
yyyyyyyyyy
3
yyyyyyyyyy
yyyyyyyyyy
4
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
5
yyyyyyyyyy
yyyyyyyyyy
6
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
7
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
8
yyyyyyyyyy
yyyyyyyyyy
9
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
10
11
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
12
yyyyyyyyyy
yyyyyyyyyy
13
yyyyyyyyyy
yyyyyyyyyy
xxxxxxxxx
yyyyyyyyyy
xxxxxxxxx
yyyyyyyyyy
14
15
16
yyyyyyyyyy
yyyyyyyyyy
17
18
19
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
20
yyyyyyyyyy
yyyyyyyyyy
21
22
xxxxxxxxx
xxxxxxxxx
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
23
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
24
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
yyyyyyyyyy
25
26
yyyyyyyyyy
yyyyyyyyyy
27
yyyyyyyy

In [ ]:
x

261

In [ ]:
np.array(HandTrackingSequences[54]).shape

(68, 126)

In [ ]:
a = 21*3*2
a

126

In [ ]:
padded_inputs1 = tf.keras.preprocessing.sequence.pad_sequences(
    HandTrackingSequences, padding="post", value=-1
)

In [ ]:
padded_inputs1.shape

(56, 93, 126)

In [ ]:
a1 = np.array(padded_inputs1)
print(np.array(padded_inputs1).shape)
a1_reshaped = a1.reshape(a1.shape[0], -1)
print(a1)
print(a1_reshaped)
np.savetxt('HandArrays.txt', a1_reshaped)
b1 = np.loadtxt('HandArrays.txt')
print('')
print(b1.shape)
b1 = b1.reshape(len(b1), len(a1[0]), a1.shape[2])
print(b1.shape)
a1 == b1
# array([ True,  True,  True,  True], dtype=bool)

(56, 93, 126)
[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 ...

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr